In [1]:
from numba import jit
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy.stats as st
from smt.sampling_methods import LHS
from datetime import datetime

In [2]:
#function that checks if our sample point is in mandelbrot set or not

def mandel_val(a,b,max_iter):   
    
    c = complex(a,b)
    z = 0.0j
    i = 0
    for i in range(max_iter):
        z = z*z + c
        if(abs(z) >= 2):
            return 0
    
    return c   

In [6]:
# MC Random Sampling with Control Variate
'''
Computationally Expensive to implement on a Intel Core i7-9750H CPU , 16 GB RAM system.
'''      

start_time = datetime.now() # Get time at start of simulation
print(f'Computation of Mandelbrot set area started at {datetime.now().strftime("%H:%M:%S")} local time.\n')

a = -2.0                         #upper limit of x & y coordinates
b = 2.0                          #lower limit of x & y coordinates
smax = 500                      # max samples drawn
imax = 5000                     # max no. of iterations
iterlist = np.arange(10,imax+10,10)
samplist = np.arange(10,smax+5,5)

def draw_randsamp(s):           #Random sampling function
    samp = []
    for i in range(s):                 
        X = a + (b-a)*random.uniform(0,1)
        Y = a + (b-a)*random.uniform(0,1)
        samp.append((X,Y))
    return samp


limit_areas = []           #areas when iterations -> imax & sample sizes -> smax
limit_areas_CV = []        # Control variate area list
limit_areas_Z = []        # New random value Z area list

for j in range(imax):       #max iterations    
    k = 0
    l = 0
    sample = draw_randsamp(smax)    #max sample size 

    for i in (sample):
        if(mandel_val(i[0], i[1], 100)!= 0):
            k = k + 1
            l = np.sqrt(k)

    limit_areas.append((k/smax)*16)
    limit_areas_CV.append((l/smax)*16)

Am = np.mean(limit_areas)         #limiting area Am
Varm = np.var(limit_areas)        #limiting Variance
intv = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas), scale=st.sem(limit_areas)) #95% confidence interval

Am_CV = np.mean(limit_areas_CV)         #limiting area Am control variate
Varm_CV = np.var(limit_areas_CV)        #limiting Variance control variate
intv_CV = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas_CV), scale=st.sem(limit_areas_CV)) #95% confidence interval control variate

mu_CV = Am_CV # = np.mean(limit_areas_CV)
cs = -np.sum((limit_areas - Am)*(limit_areas_CV - Am_CV))/np.sum((limit_areas_CV - Am_CV)**2)
limit_areas_Z = limit_areas + cs*(limit_areas_CV - mu_CV)

Am_Z = np.mean(limit_areas_Z)         #limiting area Am new random variable Z
Varm_Z = np.var(limit_areas_Z)        #limiting Variance new random variable Z
intv_Z = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas_Z), scale=st.sem(limit_areas_Z)) #95% confidence interval control variate


print(f'The computation took {datetime.now()-start_time} (HH:mm:ss) to complete.\n') # Calculate Mandelbrot area computation time
print('Limiting Area: ',Am,'\nLimiting Variance: ',Varm,'\n95% confidence interval: ',intv,'\n')
print('Limiting Area control variate: ',Am_CV,'\nLimiting Variance control variate: ',Varm_CV,'\n95% confidence interval control variate: ',intv_CV,'\n')
print('Limiting Area Z: ',Am_Z,'\nLimiting Variance Z: ',Varm_Z,'\n95% confidence interval Z: ',intv_Z,'\n')
print('Variance reduction: ',Varm/Varm_Z)

Computation of Mandelbrot set area started at 17:00:59 local time.

The computation took 0:00:09.541052 (HH:mm:ss) to complete.

Limiting Area:  1.5414527999999998 
Limiting Variance:  0.04194443657216 
95% confidence interval:  (1.535775475667874, 1.5471301243321256) 

Limiting Area control variate:  0.22159999160668845 
Limiting Variance control variate:  0.0002199333199156068 
95% confidence interval control variate:  (0.22118888728938832, 0.22201109592398857) 

Limiting Area Z:  1.5414528 
Limiting Variance Z:  9.381008827208483e-05 
95% confidence interval Z:  (1.5411843080340129, 1.5417212919659873) 

Variance reduction:  447.12074516447774


In [7]:
# MC LHS Sampling with control variate
'''
We lower the maximum samples and iterations and get lower variances and faster convergences which shows
LHS sampling is better.
'''

start_time = datetime.now() # Get time at start of simulation
print(f'Computation of Mandelbrot set area started at {datetime.now().strftime("%H:%M:%S")} local time.\n')

smax = 500                      # max samples 
imax = 5000                     # max no. of iterations
iterlist = np.arange(10,imax+10,10)
samplist = np.arange(10,smax+5,5)

def draw_lhsamp(s):             #LHS sampling function
    arr = np.array([[a, b], [a, b]])
    sampling = LHS(xlimits = arr)
    samp = sampling(s)
    return np.ndarray.tolist(samp)

limit_areas = []           #areas when iterations -> imax & sample sizes -> smax
limit_areas_CV = []        # Control variate area list
limit_areas_Z = []        # New random value Z area list

for j in range(imax):       #max iterations  
    k = 0
    l = 0
    sample = draw_lhsamp(smax)    #max sample size 

    for i in (sample):
        if(mandel_val(i[0], i[1], 100)!= 0):
            k = k + 1
            l = np.sqrt(k)

    limit_areas.append((k/smax)*16)
    limit_areas_CV.append((l/smax)*16)

Am = np.mean(limit_areas)         #limiting area Am
Varm = np.var(limit_areas)        #limiting Variance
intv = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas), scale=st.sem(limit_areas)) #95% confidence interval

Am_CV = np.mean(limit_areas_CV)         #limiting area Am control variate
Varm_CV = np.var(limit_areas_CV)        #limiting Variance control variate
intv_CV = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas_CV), scale=st.sem(limit_areas_CV)) #95% confidence interval control variate

mu_CV = Am_CV # = np.mean(limit_areas_CV)
cs = -np.sum((limit_areas - Am)*(limit_areas_CV - Am_CV))/np.sum((limit_areas_CV - Am_CV)**2)
limit_areas_Z = limit_areas + cs*(limit_areas_CV - mu_CV)

Am_Z = np.mean(limit_areas_Z)         #limiting area Am new random variable Z
Varm_Z = np.var(limit_areas_Z)        #limiting Variance new random variable Z
intv_Z = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas_Z), scale=st.sem(limit_areas_Z)) #95% confidence interval control variate

print(f'The computation took {datetime.now()-start_time} (HH:mm:ss) to complete.\n') # Calculate Mandelbrot area computation time
print('Limiting Area: ',Am,'\nLimiting Variance: ',Varm,'\n95% confidence interval: ',intv,'\n')
print('Limiting Area control variate: ',Am_CV,'\nLimiting Variance control variate: ',Varm_CV,'\n95% confidence interval control variate: ',intv_CV,'\n')
print('Limiting Area Z: ',Am_Z,'\nLimiting Variance Z: ',Varm_Z,'\n95% confidence interval Z: ',intv_Z,'\n')
print('Variance reduction: ',Varm/Varm_Z)

Computation of Mandelbrot set area started at 17:01:40 local time.

The computation took 0:00:08.949821 (HH:mm:ss) to complete.

Limiting Area:  1.5495231999999999 
Limiting Variance:  0.02561352626176 
95% confidence interval:  (1.5450866926127613, 1.5539597073872384) 

Limiting Area control variate:  0.2223768671287421 
Limiting Variance control variate:  0.00013327136600577608 
95% confidence interval control variate:  (0.2220568487304385, 0.22269688552704572) 

Limiting Area Z:  1.5495232 
Limiting Variance Z:  3.3644921250392566e-05 
95% confidence interval Z:  (1.549362407317458, 1.5496839926825423) 

Variance reduction:  761.289529291472


In [5]:
# MC Orthogonal Sampling
'''
We further lower the iterations and still get improved results which shows Orthogonal sampling is an improvement on LHS.

'''

start_time = datetime.now() # Get time at start of simulation
print(f'Computation of Mandelbrot set area started at {datetime.now().strftime("%H:%M:%S")} local time.\n')

smax = 23**2                      # max samples drawn is a perfect square to ensure we get a square matrix
imax = 5000                     # max no. of iterations
iterlist = np.arange(10,imax+10,10) 
samplist = np.array([n**2 for n in range(3,24)])   

def draw_orthosamp(samples):   #Orthogonal Sampling function 
    
    major = int(math.sqrt(samples))
    xlist = np.zeros((major,major))
    ylist = np.zeros((major,major))
    scale = 4/samples           # from -2 to 2 . 
    m = 0
    samp = []
    
    for i in range(major):
        
        for j in range (major):
            xlist[i][j] = ylist[i][j] = m
            m+=1
            
    for i in range(major):
        xlist[i] = np.random.permutation(xlist[i])
        ylist[i] = np.random.permutation(ylist[i])

    for i in range(major):

        for j in range(major):
            x = -2.0 + scale * (xlist[i][j] + np.random.random()) 
            y = -2.0 + scale * (ylist[j][i] + np.random.random())
            samp.append((x,y)) 
                
    return samp 


limit_areas = []           #areas when iterations -> imax & sample sizes -> smax
limit_areas_CV = []        # Control variate area list
limit_areas_Z = []        # New random value Z area list

for j in range(imax):       #max iterations
    k = 0
    l = 0
    sample = draw_orthosamp(smax)  #max samples
    
    for i in (sample):
        if(mandel_val(i[0], i[1], 100)!= 0):
            k = k + 1
            l = np.sqrt(k)

    limit_areas.append((k/smax)*16)
    limit_areas_CV.append((l/smax)*16)

Am = np.mean(limit_areas)         #limiting area Am
Varm = np.var(limit_areas)        #limiting Variance
intv = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas), scale=st.sem(limit_areas)) #95% confidence interval

Am_CV = np.mean(limit_areas_CV)         #limiting area Am control variate
Varm_CV = np.var(limit_areas_CV)        #limiting Variance control variate
intv_CV = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas_CV), scale=st.sem(limit_areas_CV)) #95% confidence interval control variate

mu_CV = Am_CV # = np.mean(limit_areas_CV)
cs = -np.sum((limit_areas - Am)*(limit_areas_CV - Am_CV))/np.sum((limit_areas_CV - Am_CV)**2)
limit_areas_Z = limit_areas + cs*(limit_areas_CV - mu_CV)

Am_Z = np.mean(limit_areas_Z)         #limiting area Am new random variable Z
Varm_Z = np.var(limit_areas_Z)        #limiting Variance new random variable Z
intv_Z = st.norm.interval(alpha=0.95, loc=np.mean(limit_areas_Z), scale=st.sem(limit_areas_Z)) #95% confidence interval control variate

print(f'The computation took {datetime.now()-start_time} (HH:mm:ss) to complete.\n') # Calculate Mandelbrot area computation time
print('Limiting Area: ',Am,'\nLimiting Variance: ',Varm,'\n95% confidence interval: ',intv,'\n')
print('Limiting Area control variate: ',Am_CV,'\nLimiting Variance control variate: ',Varm_CV,'\n95% confidence interval control variate: ',intv_CV,'\n')
print('Limiting Area Z: ',Am_Z,'\nLimiting Variance Z: ',Varm_Z,'\n95% confidence interval Z: ',intv_Z,'\n')
print('Variance reduction: ',Varm/Varm_Z)

Computation of Mandelbrot set area started at 16:58:51 local time.

The computation took 0:00:15.753419 (HH:mm:ss) to complete.

Limiting Area:  1.5476869565217393 
Limiting Variance:  0.005308215533249236 
95% confidence interval:  (1.5456672847436308, 1.5497066282998477) 

Limiting Area control variate:  0.21629827556445574 
Limiting Variance control variate:  2.6003633112747326e-05 
95% confidence interval control variate:  (0.2161569165405735, 0.21643963458833798) 

Limiting Area Z:  1.5476869565217386 
Limiting Variance Z:  1.3317620189167632e-06 
95% confidence interval Z:  (1.547654966109609, 1.5477189469338681) 

Variance reduction:  3985.858928134071


"\nAj_lh = []                         # areas for constant sample size\nvar_j_lh = []                      # variances for constant sample size\n\n\nfor iter in iterlist:\n    areas = []\n    \n    for j in range(iter):       #variable iterations\n        k = 0\n        sample = draw_orthosamp(7**2)    #constant sample size of 49\n   \n        for i in (sample):\n            if(mandel_val(i[0], i[1], 100)!= 0):\n                k = k + 1\n\n        areas.append((k/49)*16)\n    Aj_lh.append(np.mean(areas))\n    var_j_lh.append(np.var(areas))\n    \nAs_lh = []                           # areas for constant iterations\nvar_s_lh = []                        # variances for constant iterations\n\n\nfor s in samplist:                #variable sample sizes\n    areas = []\n    \n    for j in range(100):         #constant iterations of 100\n        k = 0\n        sample = draw_orthosamp(s)\n   \n        for i in (sample):\n            if(mandel_val(i[0], i[1], 100)!= 0):\n                k = k 